# 練習 - 股票資料彙整_Yahoo股市 - 解答

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
import datetime

目標資料來源:  
https://tw.stock.yahoo.com/s/list.php?c=tse&pid=1

## 抓取網頁資料

In [4]:
def getDataOnePage(url):
    
    targetTableIndex = 0
    table = pd.read_html(url,
                         attrs = {'border': '1' , 
                                  'cellspacing': '0', 
                                  'cellpadding': '2', 
                                  'bgcolor': '#ffffff'},
                         header = 0
                        )[targetTableIndex]
    
    return table

In [5]:
def getDataOnePageTSE(page):
    url = 'https://tw.stock.yahoo.com/s/list.php?c=tse&pid=' + str(page)   
    return   getDataOnePage(url = url)

In [16]:
# 抓第一頁的資料
df = getDataOnePageTSE(1)

In [17]:
df.tail()

,選擇,股票代號,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低,凱基證券下單
195,NaN,1729 必翔,13:30,73.0,72.9,73.0,0.00,389,73.00,73.0,73.3,72.8,買 賣 張 零股交易
196,NaN,1731 美吾華,14:30,15.80,15.8,15.85,△0.05,4068,15.75,15.60,15.80,15.15,買 賣 張 零股交易
197,NaN,1733 五鼎,13:30,44.85,44.8,44.90,△0.05,43,44.80,44.80,44.90,44.55,買 賣 張 零股交易
198,NaN,1734 杏輝,14:30,31.25,31.2,31.25,△0.15,1274,31.10,31.15,31.80,31.10,買 賣 張 零股交易
199,NaN,1736 喬山,13:30,55.0,55.0,55.1,△0.1,282,54.90,55.6,55.6,54.9,買 賣 張 零股交易


In [8]:
df.to_excel('stock.xlsx')

## 修整 DataFrame中的資料

In [18]:
def fixTable(marketType, table, theDate = datetime.date.today()):
    
    fixedTable = table
    
    # Drop
    fixedTable.drop(['選擇', '凱基證券下單'], axis = 1, inplace = True)
    fixedTable.dropna(axis=0, how='all', inplace=True)

    # fill missing data
    fixedTable['股票代號名稱'] = fixedTable['股票代號']
    fixedTable['股票代號'] = fixedTable['股票代號名稱'].map(lambda x: x.split()[0])
    fixedTable['股票名稱'] = fixedTable['股票代號名稱'].map(lambda x: x.split()[1])
    fixedTable['日期'] = theDate
    fixedTable['市場別'] = marketType
    
    # data type
    fixedTable.replace('－', np.nan, inplace = True)    
     
    fixedTable['股票代號'] = fixedTable['股票代號'].astype(str)
    fixedTable['時間'] = fixedTable['時間'].astype(datetime.time)    
    fixedTable[['成交', '買進', '賣出', '張數', '昨收', '開盤', '最高', '最低']] = \
        fixedTable[['成交', '買進', '賣出', '張數', '昨收', '開盤', '最高', '最低']].astype(float)  
    
    fixedTable['漲跌'] = fixedTable['成交'] - fixedTable['昨收']
    fixedTable['漲跌'] = fixedTable['漲跌'].map(lambda x: round(x, 2))
    
    # sort
#     fixedTable.sort_values(by = '股票代號', inplace = True) 
    
    # indexing
    fixedTable.index = Series(range(len(fixedTable)))
    fixedTable.index.name = '項次'
    fixedTable = fixedTable.reindex(columns = ['市場別', '股票代號', '股票名稱', '日期', '時間', '成交', '買進', '賣出', '漲跌', '張數', '昨收', '開盤', '最高', '最低'])
    
    return fixedTable

In [19]:
df1 = fixTable('TSE', df)

In [23]:
df1.tail(5)

,市場別,股票代號,股票名稱,日期,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低
項次,,,,,,,,,,,,,,
195,TSE,1729,必翔,2016-03-12,13:30,73.00,72.9,73.00,0.00,389,73.00,73.00,73.3,72.80
196,TSE,1731,美吾華,2016-03-12,14:30,15.80,15.8,15.85,0.05,4068,15.75,15.60,15.8,15.15
197,TSE,1733,五鼎,2016-03-12,13:30,44.85,44.8,44.90,0.05,43,44.80,44.80,44.9,44.55
198,TSE,1734,杏輝,2016-03-12,14:30,31.25,31.2,31.25,0.15,1274,31.10,31.15,31.8,31.10
199,TSE,1736,喬山,2016-03-12,13:30,55.00,55.0,55.10,0.10,282,54.90,55.60,55.6,54.90


## 彙整 Yahoo 股市 page 1~ 5 的資料

In [49]:
# 抓第一頁~第五頁的資料
dfs = map(lambda p: fixTable('TSE', getDataOnePageTSE(p)) , range(1, 6))

In [50]:
# Append 在一起
df = pd.concat(dfs)
len(df)

1000

In [52]:
df.index = pd.Index(range(len(df)))  # 重新編排 row index 編號
df = df[df['股票代號'].str.len() <= 4]  # 濾除 權證 資料
df.tail()

,市場別,股票代號,股票名稱,日期,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低
894,TSE,9941,裕融,2016-03-12,13:30,75.10,75.0,75.20,-0.30,183,75.40,75.70,75.70,74.70
895,TSE,9942,茂順,2016-03-12,13:30,89.40,89.2,89.40,0.50,122,88.90,88.20,89.80,88.10
896,TSE,9944,新麗,2016-03-12,13:30,25.30,24.9,25.30,0.00,27,25.30,25.10,25.30,24.80
897,TSE,9945,潤泰新,2016-03-12,14:30,47.20,47.2,47.25,0.95,11271,46.25,46.70,47.75,46.70
898,TSE,9955,佳龍,2016-03-12,13:30,15.05,15.0,15.05,0.65,391,14.40,14.55,15.40,14.55


## 抓取 類股 資料

In [37]:
df_類股 = pd.read_excel('..\\data\個股_類別.xls')  # 需先解壓縮 個股_類別.rar
df_類股.tail()

,市場別_ID,類股別_ID,個股_代號,個股_名稱,類股_名稱
29345,2,72,72861P,國泰RG,櫃認售
29346,2,72,72863P,國泰RJ,櫃認售
29347,2,72,72895P,元大P3,櫃認售
29348,2,72,72901P,工銀QM,櫃認售
29349,2,72,72953P,日盛QW,櫃認售


## Merge

In [53]:
mdf = df.merge(df_類股, left_on = '股票代號', right_on = '個股_代號', how = 'left')  # merge
mdf = mdf.drop(['市場別_ID', '個股_代號', '個股_名稱'], axis = 1)  # drop 多於的欄位
mdf.tail()

,市場別,股票代號,股票名稱,日期,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低,類股別_ID,類股_名稱
878,TSE,9941,裕融,2016-03-12,13:30,75.10,75.0,75.20,-0.30,183,75.40,75.70,75.70,74.70,6,其他
879,TSE,9942,茂順,2016-03-12,13:30,89.40,89.2,89.40,0.50,122,88.90,88.20,89.80,88.10,6,其他
880,TSE,9944,新麗,2016-03-12,13:30,25.30,24.9,25.30,0.00,27,25.30,25.10,25.30,24.80,6,其他
881,TSE,9945,潤泰新,2016-03-12,14:30,47.20,47.2,47.25,0.95,11271,46.25,46.70,47.75,46.70,6,其他
882,TSE,9955,佳龍,2016-03-12,13:30,15.05,15.0,15.05,0.65,391,14.40,14.55,15.40,14.55,6,其他


## GroupBy

In [56]:
# 各類股有多少支個股
mdf.groupby(['類股_名稱']).size().sort_index()

類股_名稱
光電       71
其他       46
其它電子     32
化工       26
半導體      66
塑膠       22
憑證        7
橡膠       10
水泥        7
汽車        6
油電燃氣      8
營建       48
玻璃        4
生技醫療     20
紡織       46
航運運輸     21
觀光       13
貿易百貨     11
資訊服務     13
通信網路     39
造紙        7
金融       34
鋼鐵       30
電器電纜     15
電子通路     22
電子零組件    81
電機       43
電腦週邊     60
食品       20
dtype: int64

In [60]:
# 各類股 平均股價
mdf.groupby(['類股_名稱'])['成交'].mean().sort_index()

類股_名稱
光電        57.292394
其他        57.474783
其它電子      42.019375
化工        33.268462
半導體       50.585909
塑膠        26.470000
憑證         4.358571
橡膠        34.870000
水泥        18.044286
汽車       133.266667
油電燃氣      37.387500
營建        18.950000
玻璃        10.505000
生技醫療      53.860000
紡織        32.143913
航運運輸      17.773333
觀光        69.923077
貿易百貨      40.230000
資訊服務      38.372308
通信網路      34.976154
造紙        13.552857
金融        15.497647
鋼鐵        17.430345
電器電纜      10.736000
電子通路      31.622857
電子零組件     37.101111
電機        63.737907
電腦週邊      47.115833
食品        31.715000
Name: 成交, dtype: float64